In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2240767,2022-03-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2240768,2022-03-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2240769,2022-03-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2240770,2022-03-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
57643,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
57645,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
57647,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
57649,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
57651,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1357511,2022-03-22,Arkansas,Arkansas,5001,5673,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1357513,2022-03-23,Arkansas,Arkansas,5001,5674,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1357515,2022-03-24,Arkansas,Arkansas,5001,5674,78.00,5000,Arkansas,AR,Arkansas,State,3017804
1357517,2022-03-25,Arkansas,Arkansas,5001,5676,78.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2240767,2022-03-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2240768,2022-03-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2240769,2022-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2240770,2022-03-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-03-26') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
795,2022-03-26,Snohomish,Washington,148815,1110.00,53061,WA,County,822083,18102.19,135.02
1588,2022-03-26,Cook,Illinois,1124384,14793.00,17031,IL,County,5150233,21831.71,287.23
2380,2022-03-26,Orange,California,586528,6860.00,6059,CA,County,3175692,18469.30,216.02
3171,2022-03-26,Maricopa,Arizona,1260012,16393.00,4013,AZ,County,4485414,28091.32,365.47
3962,2022-03-26,Los Angeles,California,2830752,31594.00,6037,CA,County,10039107,28197.25,314.71
...,...,...,...,...,...,...,...,...,...,...,...
2238775,2022-03-26,Wheeler,Oregon,242,3.00,41069,OR,County,1332,18168.17,225.23
2239305,2022-03-26,King,Texas,41,0.00,48269,TX,County,272,15073.53,0.00
2239804,2022-03-26,Esmeralda,Nevada,109,3.00,32009,NV,County,873,12485.68,343.64
2240299,2022-03-26,Loving,Texas,170,1.00,48301,TX,County,169,100591.72,591.72


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
795,2022-03-26,Snohomish,Washington,148815,1110.00,53061,WA,County,822083,18102.19,135.02,135.02,18102.19
1588,2022-03-26,Cook,Illinois,1124384,14793.00,17031,IL,County,5150233,21831.71,287.23,287.23,21831.71
2380,2022-03-26,Orange,California,586528,6860.00,6059,CA,County,3175692,18469.30,216.02,216.02,18469.30
3171,2022-03-26,Maricopa,Arizona,1260012,16393.00,4013,AZ,County,4485414,28091.32,365.47,365.47,28091.32
3962,2022-03-26,Los Angeles,California,2830752,31594.00,6037,CA,County,10039107,28197.25,314.71,314.71,28197.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238775,2022-03-26,Wheeler,Oregon,242,3.00,41069,OR,County,1332,18168.17,225.23,225.23,18168.17
2239305,2022-03-26,King,Texas,41,0.00,48269,TX,County,272,15073.53,0.00,0.00,15073.53
2239804,2022-03-26,Esmeralda,Nevada,109,3.00,32009,NV,County,873,12485.68,343.64,343.64,12485.68
2240299,2022-03-26,Loving,Texas,170,1.00,48301,TX,County,169,100591.72,591.72,591.72,100591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1420124,2022-03-26,Hale,Alabama,4697,104.00,1065,AL,County,14651,32059.25,709.85,709.85,32059.25,1
1197758,2022-03-26,Winston,Alabama,7569,128.00,1133,AL,County,23629,32032.67,541.71,541.71,32032.67,2
822816,2022-03-26,Franklin,Alabama,9851,137.00,1059,AL,County,31362,31410.62,436.83,436.83,31410.62,3
979023,2022-03-26,Clay,Alabama,4085,81.00,1027,AL,County,13235,30865.13,612.01,612.01,30865.13,4
1419397,2022-03-26,Fayette,Alabama,4972,94.00,1057,AL,County,16302,30499.33,576.62,576.62,30499.33,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917684,2022-03-26,Big Horn,Wyoming,2575,60.00,56003,WY,County,11790,21840.54,508.91,508.91,21840.54,19
1194108,2022-03-26,Johnson,Wyoming,1844,19.00,56019,WY,County,8445,21835.41,224.99,224.99,21835.41,20
1688451,2022-03-26,Lincoln,Wyoming,3966,30.00,56023,WY,County,19830,20000.00,151.29,151.29,20000.00,21
1355335,2022-03-26,Sublette,Wyoming,1923,27.00,56035,WY,County,9831,19560.57,274.64,274.64,19560.57,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1108581,2022-03-26,Lowndes,Alabama,2609,75.00,1085,AL,County,9726,26825.01,771.13,771.13,26825.01,31,1
1107119,2022-03-26,Crenshaw,Alabama,3787,98.00,1041,AL,County,13772,27497.82,711.59,711.59,27497.82,25,2
1420124,2022-03-26,Hale,Alabama,4697,104.00,1065,AL,County,14651,32059.25,709.85,709.85,32059.25,1,3
468520,2022-03-26,Walker,Alabama,18536,436.00,1127,AL,County,63521,29180.90,686.39,686.39,29180.90,10,4
976827,2022-03-26,Butler,Alabama,5048,127.00,1013,AL,County,19448,25956.40,653.02,653.02,25956.40,35,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194108,2022-03-26,Johnson,Wyoming,1844,19.00,56019,WY,County,8445,21835.41,224.99,224.99,21835.41,20,19
1645794,2022-03-26,Uinta,Wyoming,5632,39.00,56041,WY,County,20226,27845.35,192.82,192.82,27845.35,5,20
1688451,2022-03-26,Lincoln,Wyoming,3966,30.00,56023,WY,County,19830,20000.00,151.29,151.29,20000.00,21,21
1104195,2022-03-26,Albany,Wyoming,9869,48.00,56001,WY,County,38880,25383.23,123.46,123.46,25383.23,14,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,135.02,18102.19,30,20
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,135.02,18102.19,30,20
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,135.02,18102.19,30,20
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,135.02,18102.19,30,20
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,135.02,18102.19,30,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235601,2022-03-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2235602,2022-03-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2235603,2022-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2235604,2022-03-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1414232,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,709.85,32059.25,3,1,1.00,nan
1414233,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,709.85,32059.25,3,1,0.00,nan
1414234,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,709.85,32059.25,3,1,0.00,nan
1414235,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,709.85,32059.25,3,1,0.00,nan
1414236,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,709.85,32059.25,3,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717215,2022-03-22,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17747.89,9,23,0.00,0.00
1717216,2022-03-23,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17747.89,9,23,0.00,0.00
1717217,2022-03-24,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17747.89,9,23,0.00,0.00
1717218,2022-03-25,Crook,Wyoming,1346,28.00,56011,WY,County,7584,17747.89,369.20,369.20,17747.89,9,23,0.00,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1104143,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26825.01,1,31,1.00,1.14,0.00,nan
1104144,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26825.01,1,31,0.00,0.57,0.00,nan
1104145,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26825.01,1,31,0.00,0.57,0.00,nan
1104146,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26825.01,1,31,0.00,0.57,0.00,nan
1104147,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,771.13,26825.01,1,31,0.00,0.43,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463326,2022-03-22,Teton,Wyoming,9887,16.00,56039,WY,County,23464,42136.89,68.19,68.19,42213.60,23,1,24.00,4.86,0.00,0.00
463327,2022-03-23,Teton,Wyoming,9887,16.00,56039,WY,County,23464,42136.89,68.19,68.19,42213.60,23,1,0.00,4.43,0.00,0.00
463328,2022-03-24,Teton,Wyoming,9887,16.00,56039,WY,County,23464,42136.89,68.19,68.19,42213.60,23,1,0.00,3.93,0.00,0.00
463329,2022-03-25,Teton,Wyoming,9905,16.00,56039,WY,County,23464,42213.60,68.19,68.19,42213.60,23,1,18.00,4.50,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-03-26') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
560000,2022-03-26,Imperial,California,66362,912.00,6025,CA,County,181215,36620.59,503.27,503.27,36620.59,1,1,0.00,42.86,0.00,1.21
1355993,2022-03-26,Tuolumne,California,13454,181.00,6109,CA,County,54478,24696.21,332.24,332.24,24696.21,2,14,0.00,7.50,0.00,0.07
85370,2022-03-26,Shasta,California,36723,587.00,6089,CA,County,180080,20392.60,325.97,325.97,20392.60,3,28,0.00,9.50,0.00,1.00
3962,2022-03-26,Los Angeles,California,2830752,31594.00,6037,CA,County,10039107,28197.25,314.71,314.71,28197.25,4,5,957.00,1298.86,18.00,22.79
277542,2022-03-26,San Bernardino,California,585213,6840.00,6071,CA,County,2180085,26843.59,313.75,313.75,26843.59,5,8,0.00,145.36,0.00,7.57
984831,2022-03-26,Inyo,California,4543,56.00,6027,CA,County,18039,25184.32,310.44,310.44,25184.32,6,11,0.00,0.43,0.00,0.00
123481,2022-03-26,Stanislaus,California,136645,1694.00,6099,CA,County,550660,24814.77,307.63,307.63,24814.77,7,13,0.00,0.00,0.00,0.00
153318,2022-03-26,Tulare,California,132835,1400.00,6107,CA,County,466195,28493.44,300.30,300.30,28493.44,8,4,0.00,50.64,0.00,3.00
1644966,2022-03-26,Tehama,California,14643,195.00,6103,CA,County,65084,22498.62,299.61,299.61,22498.62,9,20,0.00,5.14,0.00,2.57
891757,2022-03-26,Merced,California,71842,810.00,6047,CA,County,277680,25872.23,291.70,291.70,25872.23,10,9,0.00,38.00,0.00,0.71


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
560000,2022-03-26,Imperial,California,66362,912.00,6025,CA,County,181215,36620.59,503.27,503.27,36620.59,1,1,0.00,42.86,0.00,1.21
1196240,2022-03-26,Kings,California,55018,444.00,6031,CA,County,152940,35973.58,290.31,290.31,35973.58,11,2,0.00,19.07,0.00,0.57
2108005,2022-03-26,Lassen,California,10710,68.00,6035,CA,County,30573,35030.91,222.42,222.42,35030.91,21,3,14.00,1.50,0.00,0.00
153318,2022-03-26,Tulare,California,132835,1400.00,6107,CA,County,466195,28493.44,300.30,300.30,28493.44,8,4,0.00,50.64,0.00,3.00
3962,2022-03-26,Los Angeles,California,2830752,31594.00,6037,CA,County,10039107,28197.25,314.71,314.71,28197.25,4,5,957.00,1298.86,18.00,22.79
56142,2022-03-26,Madera,California,43360,354.00,6039,CA,County,157327,27560.43,225.01,225.01,27560.43,20,6,0.00,8.71,0.00,0.00
349461,2022-03-26,Kern,California,242196,2265.00,6029,CA,County,900202,26904.63,251.61,251.61,26904.63,16,7,0.00,92.29,0.00,5.93
277542,2022-03-26,San Bernardino,California,585213,6840.00,6071,CA,County,2180085,26843.59,313.75,313.75,26843.59,5,8,0.00,145.36,0.00,7.57
891757,2022-03-26,Merced,California,71842,810.00,6047,CA,County,277680,25872.23,291.70,291.70,25872.23,10,9,0.00,38.00,0.00,0.71
55392,2022-03-26,Fresno,California,252053,2710.00,6019,CA,County,999101,25227.98,271.24,271.24,25227.98,13,10,0.00,144.14,0.00,4.07


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
560000,2022-03-26,Imperial,California,66362,912.00,6025,CA,County,181215,36620.59,503.27,503.27,36620.59,1,1,0.00,42.86,0.00,1.21
1355993,2022-03-26,Tuolumne,California,13454,181.00,6109,CA,County,54478,24696.21,332.24,332.24,24696.21,2,14,0.00,7.50,0.00,0.07
85370,2022-03-26,Shasta,California,36723,587.00,6089,CA,County,180080,20392.60,325.97,325.97,20392.60,3,28,0.00,9.50,0.00,1.00
3962,2022-03-26,Los Angeles,California,2830752,31594.00,6037,CA,County,10039107,28197.25,314.71,314.71,28197.25,4,5,957.00,1298.86,18.00,22.79
277542,2022-03-26,San Bernardino,California,585213,6840.00,6071,CA,County,2180085,26843.59,313.75,313.75,26843.59,5,8,0.00,145.36,0.00,7.57
984831,2022-03-26,Inyo,California,4543,56.00,6027,CA,County,18039,25184.32,310.44,310.44,25184.32,6,11,0.00,0.43,0.00,0.00
123481,2022-03-26,Stanislaus,California,136645,1694.00,6099,CA,County,550660,24814.77,307.63,307.63,24814.77,7,13,0.00,0.00,0.00,0.00
153318,2022-03-26,Tulare,California,132835,1400.00,6107,CA,County,466195,28493.44,300.30,300.30,28493.44,8,4,0.00,50.64,0.00,3.00
1644966,2022-03-26,Tehama,California,14643,195.00,6103,CA,County,65084,22498.62,299.61,299.61,22498.62,9,20,0.00,5.14,0.00,2.57
891757,2022-03-26,Merced,California,71842,810.00,6047,CA,County,277680,25872.23,291.70,291.70,25872.23,10,9,0.00,38.00,0.00,0.71


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,1,4.00,1.93
1,03/21/20,Imperial,4,2.21,1,0.00,1.93
2,03/22/20,Imperial,4,2.21,1,0.00,1.64
3,03/23/20,Imperial,4,2.21,1,0.00,1.57
4,03/24/20,Imperial,9,4.97,1,5.00,1.93
...,...,...,...,...,...,...,...
7386,03/22/22,Lassen,10696,34985.12,3,0.00,0.93
7387,03/23/22,Lassen,10696,34985.12,3,0.00,0.71
7388,03/24/22,Lassen,10696,34985.12,3,0.00,0.71
7389,03/25/22,Lassen,10696,34985.12,3,0.00,0.57


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.07
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.07
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.07
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.07
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.07
...,...,...,...,...,...,...,...
7420,03/22/22,Merced,810.00,291.70,10,0.00,1.00
7421,03/23/22,Merced,810.00,291.70,10,0.00,0.93
7422,03/24/22,Merced,810.00,291.70,10,0.00,0.71
7423,03/25/22,Merced,810.00,291.70,10,0.00,0.71


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)